<a href="https://colab.research.google.com/github/LucasbFrancaa/HDATA/blob/main/CASE1_HDATA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.cloud import storage
!pip install google-cloud-bigquery
!pip install apache_beam
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.6/255.6 kB 15.7 MB/s eta 0:00:00
  

#Configs

In [ ]:
# Exemplo de uso
bucket_name = ''
source_file_name = ''  # Arquivo local
destination_blob_name = ''

# 1. Entendimento do Processo

In [7]:
dados_recebidos= pd.read_excel('/content/drive/MyDrive/case1/Dados Financeiros - Dados Recebidos.xlsx')
estrutura_hdata= pd.read_excel('/content/drive/MyDrive/case1/Dados Financeiros - Estrutura HData.xlsx')

In [6]:
# Agora, vamos mapear os dados recebidos com a estrutura HData
# Vamos criar um mapeamento baseado na coluna "Coluna Planilha" da estrutura HData para as colunas nos dados recebidos

# Criar um dicionário de mapeamento de aderência baseado no layout
mapeamento_colunas = dict(zip(estrutura_hdata['Coluna Planilha'], estrutura_hdata['CAMPO']))

# Aplicar o mapeamento nos dados recebidos para renomear as colunas
dados_enriquecidos = dados_recebidos.rename(columns=mapeamento_colunas)

# Exibir o resultado do mapeamento para ver as primeiras colunas renomeadas
dados_enriquecidos.head()


,Identificador ou código único do lançamento,Conta,Centro de Custos,Mês de Referência/competência da prestação de serviço,Vencimento,Fornecedor ou Prestador de Serviço (Descrição),Identificador do Documento ou Nota Fiscal,Valor da Nota Fiscal (Valor Bruto),Valor da Parcela,Considerações (histórico ou observação),Situação,Data Realização,Valor Realizado,Conta bancária,Forma de Pagamento
0,6.0,MATERIAL DESCARTAVEL (CONSUMO DE MATERIAIS GER...,Farmacia,2023-03-01,2023-04-28,FORNECEDOR 444,18185,15.00,15.00,Materiais Descartaveis Cozinha,PAGO,2023-04-01,15.00,GERAL,NaN
1,7.0,PECAS E MATERIAIS DE MANUTENCAO - PREDIAL (MAN...,Tecnologia Informacao,2023-03-01,2023-03-10,FORNECEDOR 183,136874,36.53,36.53,Material de manutencao predial,PAGO,2023-03-01,36.53,GERAL,NaN
2,8.0,PECAS E MATERIAIS DE MANUTENCAO - PREDIAL (MAN...,Ambulatorio,2023-03-01,2023-03-27,FORNECEDOR 183,137186,328.58,328.58,Material de manutencao predial,PAGO,2023-03-01,328.58,GERAL,NaN
3,9.0,PECAS E MATERIAIS DE MANUTENCAO - PREDIAL (MAN...,UTI,2023-03-01,2023-04-24,FORNECEDOR 183,137186,328.58,328.58,Material de manutencao predial,PAGO,2023-04-01,328.58,GERAL,NaN
4,10.0,PECAS E MATERIAIS DE MANUTENCAO - PREDIAL (MAN...,Ambulatorio,2023-03-01,2023-05-22,FORNECEDOR 183,137186,328.58,328.58,Fechadura,PAGO,2023-05-01,328.58,GERAL,NaN


#1.Upload de um arquivo para o GCS

In [ ]:
# Função para fazer upload de um arquivo para o GCS
def upload_to_gcs(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)
    print(f"File {source_file_name} uploaded to {destination_blob_name}.")

# Função para baixar um arquivo do GCS
def download_from_gcs(bucket_name, source_blob_name, destination_file_name):
    """Downloads a file from the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)
    print(f"File {source_blob_name} downloaded to {destination_file_name}.")

#2. Armazenamento de Dados no Google Cloud Storage (GCS)

In [ ]:
def process_financial_data(event, context):
    bucket_name = event['bucket']
    file_name = event['name']

    # Conectar ao GCS e baixar o arquivo carregado
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(file_name)
    temp_local_file = f'/tmp/{file_name}'
    blob.download_to_filename(temp_local_file)

    # Ler o arquivo com pandas
    df = pd.read_excel(temp_local_file)

    # Enriquecimento de dados (exemplo: aplicar desconto de 10%)
    df['Valor com Desconto'] = df['Valor da Nota Fiscal (Valor Bruto)'] * 0.9

    # Exportar o arquivo enriquecido de volta para o GCS
    enriched_file_name = 'enriched_' + file_name
    enriched_blob = bucket.blob(enriched_file_name)
    df.to_csv(f'/tmp/{enriched_file_name}', index=False)
    enriched_blob.upload_from_filename(f'/tmp/{enriched_file_name}')

    print(f"Arquivo {file_name} processado e enriquecido como {enriched_file_name}")


#3. Carregamento para o BigQuery

In [ ]:
def load_to_bigquery(dataset_id, table_id, source_uri):
    client = bigquery.Client()
    table_ref = client.dataset(dataset_id).table(table_id)

    job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.CSV, skip_leading_rows=1, autodetect=True
    )

    load_job = client.load_table_from_uri(source_uri, table_ref, job_config=job_config)
    load_job.result()  # Aguarda a conclusão do carregamento
    print(f"Dados carregados na tabela {table_id}")


#Estrutura de Data Warehouse com BigQuery


No BigQuery podemos seguir uma estrutura de Esquema Estrela (Star Schema).
---



In [7]:
CREATE TABLE `hospital_dw.Fato_Financeiro` (
    id_lancamento INT64,
    data_vencimento DATE,
    valor_bruto FLOAT64,
    valor_com_desconto FLOAT64,
    id_fornecedor INT64,
    id_conta INT64,
    PRIMARY KEY(id_lancamento)
);

CREATE TABLE `hospital_dw.Dim_Fornecedor` (
    id_fornecedor INT64,
    nome STRING,
    PRIMARY KEY(id_fornecedor)
);


In [ ]:
from google.cloud import bigquery

# Função para carregar os dados para o BigQuery
def load_to_bigquery(dataset_id, table_id, source_uri):
    client = bigquery.Client()
    table_ref = client.dataset(dataset_id).table(table_id)

    job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.CSV, skip_leading_rows=1, autodetect=True
    )

    load_job = client.load_table_from_uri(source_uri, table_ref, job_config=job_config)
    load_job.result()  # Espera até que a job termine
    print(f"Carregamento para BigQuery concluído. Tabela: {dataset_id}.{table_id}")

# Exemplo de uso:
bucket_name = 'nome-do-seu-bucket'
enriched_file_gcs_uri = f'gs://{bucket_name}/enriched/dados_enriquecidos_no_gcs.csv'
dataset_id = 'seu_dataset'
table_id = 'tabela_enriquecida'

load_to_bigquery(dataset_id, table_id, enriched_file_gcs_uri)


#Orquestração de Pipelines com Dataflow

In [ ]:


def enrich_row(row):
    # Enriquecimento da linha de dados
    row['valor_com_desconto'] = row['valor'] * 0.9
    return row

def run():
    pipeline_options = PipelineOptions(
        project='seu-projeto',
        runner='DataflowRunner',
        temp_location='gs://seu-bucket/temp',
        region='us-central1'
    )

    with beam.Pipeline(options=pipeline_options) as p:
        (p
         | 'ReadCSV' >> beam.io.ReadFromText('gs://seu-bucket/dados.csv', skip_header_lines=1)
         | 'ParseCSV' >> beam.Map(lambda line: dict(zip(['id', 'valor', 'categoria', 'data_transacao'], line.split(','))))
         | 'EnrichData' >> beam.Map(enrich_row)
         | 'WriteToCSV' >> beam.io.WriteToText('gs://seu-bucket/enriched_dados', file_name_suffix='.csv'))

if __name__ == '__main__':
    run()


#*Bonus* Resposta
 ideia é que a Cloud Function seja acionada automaticamente sempre que um novo arquivo for enviado ao Google Cloud Storage e dependendo do layout do arquivo, aplique regras de enriquecimento específicas.

Quando um arquivo é enviado para um bucket no GCS, a função é automaticamente acionada, processa o arquivo e aplica as transformações necessárias.